In [15]:
# from google.colab import drive
# drive.mount('/content/drive/')

# %cd /content/drive/MyDrive/IT5006_code/

In [16]:
import torch
import numpy as np
from torch import optim as optim
from cmapss import cmapss
from torch.utils.data import DataLoader
from model import Model, Model_CNN
from train import Trainer
from tqdm import tqdm

## GPU Check

In [17]:
import torch

if torch.cuda.is_available():
    print("GPU available")
    print("GPU name:", torch.cuda.get_device_name(0))

In [18]:
sets = ['FD001', 'FD002', 'FD003', 'FD004']
# sets = ['FD001']
model_name = 'CNN_feature'
# feature_extract_mode = 'default'
feature_extract_mode = 'tsfresh'

iteration_num = 2
window_size = 30
ori_feature_num = 17

In [19]:
model_class = {
    'CNN': Model_CNN,
    'CNN_feature': Model_CNN_feature,
    # 'LSTM': Model_LSTM,
    # 'LSTM_feature': Model_LSTM_feature,
    'Hybrid': Model
}

In [20]:
def run(iteration_num=10, window_size=30, set_name='FD001', model_name='CNN', feature_extract_mode='default'):

    trainset = cmapss(mode='train',
                    feature_extract_mode=feature_extract_mode,
                    dataset=f'./CMAPSSData/train_{set_name}_processed.csv',
                    window_size=window_size)
    testset = cmapss(mode='test',
                    feature_extract_mode=feature_extract_mode,
                    dataset=f'./CMAPSSData/test_{set_name}_processed.csv',
                    rul_result=f'./CMAPSSData/RUL_{set_name}.txt',
                    window_size=window_size)

    handcrafted_feature_num = trainset.feature_extract().shape[1]
    testset.feature_extract()
    print(f'Feature extraction get {handcrafted_feature_num} handcrafted features')

    train_loader = DataLoader(dataset=trainset, batch_size=100, shuffle=True, num_workers=2)
    test_loader = DataLoader(dataset=testset, batch_size=64, shuffle=False, num_workers=2)
    print('Dataset load successfully')

    best_score_list = []
    best_RMSE_list = []
    
    result_name = f'{set_name}_{model_name}_{feature_extract_mode}'

    for iteration in range(iteration_num):
        print('---Iteration: {}---'.format(iteration + 1))
        model = model_class[model_name](window_size, ori_feature_num, handcrafted_feature_num)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        epochs = 32
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        trainer = Trainer(model=model,
                            model_optimizer=optimizer,
                            print_every=20,
                            epochs=epochs,
                            device=device,
                            result_name=result_name)
        best_score, best_RMSE = trainer.train(train_loader, test_loader, iteration)
        best_score_list.append(best_score)
        best_RMSE_list.append(best_RMSE)

    best_score_list = np.array(best_score_list)
    best_RMSE_list = np.array(best_RMSE_list)
    result = np.concatenate((best_score_list, best_RMSE_list)).reshape(2, iteration_num)
    np.savetxt(f'./result/{result_name}_result.txt', result, fmt='%.4f')

In [21]:
for set_name in sets:
    run(iteration_num = iteration_num, 
        window_size = window_size,
        set_name = set_name,
        model_name = model_name,
        feature_extract_mode = feature_extract_mode)

Run tsfresh feature extraction for train dataset


Processing units: 100%|██████████| 17731/17731 [00:14<00:00, 1218.41unit/s]


Run tsfresh feature extraction for test dataset


Processing units: 100%|██████████| 100/100 [00:00<00:00, 1193.72unit/s]


Feature extraction get 47 handcrafted features
Dataset load successfully
---Iteration: 1---
Epoch: 1
batch:20/178, loss(avg. on 20 batches: 0.27999507933855056
batch:40/178, loss(avg. on 20 batches: 0.04984724149107933
batch:60/178, loss(avg. on 20 batches: 0.03743836898356676
batch:80/178, loss(avg. on 20 batches: 0.03415915835648775
batch:100/178, loss(avg. on 20 batches: 0.03255183640867472
batch:120/178, loss(avg. on 20 batches: 0.0293897015042603
batch:140/178, loss(avg. on 20 batches: 0.030236897245049476
batch:160/178, loss(avg. on 20 batches: 0.028485788963735104
test result: score: 3507.056640625, RMSE: 22.19875144958496
Epoch: 2
batch:20/178, loss(avg. on 20 batches: 0.02539579141885042
batch:40/178, loss(avg. on 20 batches: 0.02377082882449031
batch:60/178, loss(avg. on 20 batches: 0.021784948837012052
batch:80/178, loss(avg. on 20 batches: 0.02096817148849368
batch:100/178, loss(avg. on 20 batches: 0.021964828204363584
batch:120/178, loss(avg. on 20 batches: 0.0210631201509

...

Feature extraction get 47 handcrafted features
Dataset load successfully
---Iteration: 1---
Epoch: 1
batch:20/219, loss(avg. on 20 batches: 0.39663059674203394
batch:40/219, loss(avg. on 20 batches: 0.0832001943141222
batch:60/219, loss(avg. on 20 batches: 0.043827814236283305
batch:80/219, loss(avg. on 20 batches: 0.03576154606416822
batch:100/219, loss(avg. on 20 batches: 0.0324460762552917
batch:120/219, loss(avg. on 20 batches: 0.032441396266222
batch:140/219, loss(avg. on 20 batches: 0.02855559755116701
batch:160/219, loss(avg. on 20 batches: 0.028952357172966004
batch:180/219, loss(avg. on 20 batches: 0.02980716684833169
batch:200/219, loss(avg. on 20 batches: 0.026559406891465187
test result: score: 3745.129150390625, RMSE: 26.100507736206055
Epoch: 2
batch:20/219, loss(avg. on 20 batches: 0.0269134059548378
batch:40/219, loss(avg. on 20 batches: 0.026345071010291578
batch:60/219, loss(avg. on 20 batches: 0.025652118120342494
batch:80/219, loss(avg. on 20 batches: 0.024531284905

Processing units: 100%|██████████| 54028/54028 [01:55<00:00, 465.92unit/s] 


Run tsfresh feature extraction for test dataset


Processing units: 100%|██████████| 248/248 [00:00<00:00, 1198.01unit/s]


Feature extraction get 47 handcrafted features
Dataset load successfully
---Iteration: 1---
Epoch: 1
batch:20/541, loss(avg. on 20 batches: 0.260116932541132
batch:40/541, loss(avg. on 20 batches: 0.09207144938409328
batch:60/541, loss(avg. on 20 batches: 0.062336158193647864
batch:80/541, loss(avg. on 20 batches: 0.053608879633247855
batch:100/541, loss(avg. on 20 batches: 0.04959701895713806
batch:120/541, loss(avg. on 20 batches: 0.05004282854497433
batch:140/541, loss(avg. on 20 batches: 0.044982445612549785
batch:160/541, loss(avg. on 20 batches: 0.03916029278188944
batch:180/541, loss(avg. on 20 batches: 0.03862628387287259
batch:200/541, loss(avg. on 20 batches: 0.035586319584399465
batch:220/541, loss(avg. on 20 batches: 0.03668710710480809
batch:240/541, loss(avg. on 20 batches: 0.034202548861503604
batch:260/541, loss(avg. on 20 batches: 0.037532161269336936
batch:280/541, loss(avg. on 20 batches: 0.035424089338630435
batch:300/541, loss(avg. on 20 batches: 0.0354954643175005